<h1 align="center" style='color:#a00e67'>An Introduction to Graph Databases with GQLAlchemy and Python</h1>

# Table of contents <a name="toc"></a>
1. [Introduction](#introduction)
2. [Prerequisites](#prerequisites)
    1. [Memgraph installation using Docker](#memgraph-docker)
3. [Dataset and graph model](#dataset)
4. [Connect to Memgraph with GQLAlchemy](#connect)
5. [Define a graph schema](#graph-schema)
6. [Creating and returning nodes and relationships](#create-return)
7. [Importing data from CSV files](#import)
8. [Querying the database and retrieving results](#query)
9. [Exercises](#exercises)

## 1. Introduction <a name="introduction"></a>

Through this tutorial, you will learn how to create a graph model from a dataset, how to run Memgraph with Docker, connect to it from a Jupyter Notebook and perform simple queries. We will explore the **movies dataset** that consists of two CSV files: `movies.csv` containing information about movie title and genre, and `ratings.csv` that holds information about users' ratings.


## 2. Prerequisites <a name="prerequisites"></a>

For this tutorial, you will need to install:

- [Jupyter](https://jupyter.org/install)
- [Docker](https://docs.docker.com/get-docker/)
- [GQLAlchemy](https://pypi.org/project/gqlalchemy/)
- [Pandas](https://pypi.org/project/pandas/)

Docker is used because Memgraph is a native Linux application and cannot be installed on Windows and macOS.

### A. Memgraph installation using Docker <a name="memgraph-docker"></a>

After you install Docker, you can set up **Memgraph** by running:

```
docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform
```

This command will start the download and after it finishes, run the Memgraph container.

**Memgraph Platform** contains:

- **Memgraph DB** - the database that holds your data
- **Memgraph Lab** - visual user interface for running queries and visualizing graph data (running at `localhost:3000`)
- **mgconsole** - command-line interface for running queries
- **MAGE** - graph algorithms and modules library



## 3. Dataset and graph model <a name="dataset"></a>

We will use pandas library to show how `movies.csv` and `ratings.csv` files look like.

In [1]:
import pandas as pd

movies = pd.read_csv("data/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


*** 

Next, we have to create a **graph model** for the movies dataset. Every movie has a title and genre it belongs to. Therefore, let's create `:Movie` and `:Genre` nodes and the relationship between them of `OF_GENRE` type. The `:Movie` nodes will have `id` and `title` properties, while `:Genre` nodes will have just `name` property. In `ratings.csv` we have users who rated movies in a certain moment. Hence, we should create `:User` nodes and new relationship type called `RATED` between `:User` and `:Movie` nodes. This relationship will have properties `rating` and `timestamp`. `:User` node only has `id` property. At the image below, you can see how we can draw the graph model (schema).

![movies_graph_model](img/movie-schema.png)

## 4. Connect to Memgraph with GQLAlchemy <a name="connect"></a>

We will be using the **GQLAlchemy** object graph mapper (OGM) to connect to Memgraph and execute **Cypher** queries easily. GQLAlchemy also serves as a Python driver/client for Memgraph. You can install it using:

```
pip install gqlalchemy
```

> **Hint**: You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.

In [3]:
from gqlalchemy import Memgraph

In [4]:
memgraph = Memgraph("127.0.0.1", 7687)

Let's make sure that Memgraph is empty before we start with anything else.

In [5]:
memgraph.drop_database()

In [6]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))


{'number_of_nodes': 0}


## 5. Define a graph schema <a name="graph-schema"></a>

Next, we have to define the above graph schema in our Python code. This is where **GQLAlchemy** steps in with its Object Graph Mapper.

In [7]:
from typing import Optional
from gqlalchemy import Node, Relationship, Field

class Movie(Node):
    id: int = Field(index=True, unique=True, exists=True, db=memgraph)
    title: Optional[str] = Field()
        

class User(Node):
    id: int = Field(index=True, unique=True, exists=True, db=memgraph)
    name: Optional[str] = Field()
        
class Genre(Node):
    name: str = Field()

class Rated(Relationship, type="RATED"):
    rating: float = Field()
    timestamp: float = Field()

class OfGenre(Relationship, type="OF_GENRE"):
    pass

What’s going on here:

- `Node` is a Python class that maps to a graph object in `Memgraph`.
- Classes that inherit from `Node` map to a single label in the graph database.
- In this case, the class `Movie` maps to the label `:Movie`.
- The properties `id` and `title` are mapped to properties of the nodes labeled `Movie` in the graph database, their types must be defined and are enforced.
- If the type of the property is missing the `Optional` keyword, then it can’t be `None` or missing.
- `Field` is a function from pydantic that you can use to define constraints in the graph database like uniqueness constraints, indexes and existence constraints.
- Whenever you provide a constraint to the `Field` function, you have to provide a `Memgraph` object in the `db` argument as well, so those constraints can be enforced.
- In this instance, a uniqueness and exists constraints, as well as index are added to `Memgraph` on the label `Movie` and the property `id`.

## 6. Creating and returning nodes and relationships <a name="create-return"></a>

Before we import the whole dataset from the CSV files, let's see how we can create a simple node and relationship. First we are going to create one `User` node and one `Movie` node.

In [8]:
user_node = User(id=1, name="Katarina")
movie_node = Movie(id=1, title="LOTR")

user_node.save(memgraph)
movie_node.save(memgraph)

print(user_node)
print(movie_node)

<User id=346615 labels={'User'} properties={'id': 1, 'name': 'Katarina'}>
<Movie id=346616 labels={'Movie'} properties={'id': 1, 'title': 'LOTR'}>


Now, let's try to create a node using the Cypher query language. We are going to create an user with an existing `id` just to check if the uniqueness constraint on the property `id` is set correctly.

In [9]:
try:
    results = memgraph.execute(
        """
        CREATE (:User {name: "Katarina"});
        """
    )
except Exception:
    print("Error: A user with the name Katarina is already in the database.")

Error: A user with the name Katarina is already in the database.


Next, let's create a relationship of type `RATED` between the existing user and movie nodes.

In [10]:
from time import time

rated_relationship = Rated(_start_node_id=user_node._id, _end_node_id=movie_node._id, rating=5.0, timestamp=time())

rated_relationship.save(memgraph)

print(rated_relationship)

<Rated id=563845 start_node_id=346615 end_node_id=346616 nodes=(346615, 346616) type=RATED properties={'rating': 5.0, 'timestamp': 1648232875.278028}>


## 7. Importing data from CSV files <a name="import"></a>

First, let's clean up the database, in order to import only data from the CSV files.


In [11]:
memgraph.drop_database()

First, we need to download the `movies.csv` and `ratings.csv` files. To import it into Memgraph, we need to copy it to the Docker container where Memgraph is running.
Find the `CONTAINER_ID` by running:

```
docker ps
```

Copy the files with the following commands (don't forget to replace `CONTAINER_ID`):

```
docker cp movies.csv CONTAINER_ID:movies.csv
```

```
docker cp ratings.csv CONTAINER_ID:ratings.csv
```

We will import data to Memgraph using the `LOAD CSV` Cypher clause that enables us to load and use data from a CSV file of your choosing in a row-based manner within a query. Memgraph supports the Excel CSV dialect, as it's the most common one. For more info, check out [Memgraph Docs](https://memgraph.com/docs/memgraph/import-data/load-csv-clause).

The following method will run `LOAD CSV` clause inside Memgraph and create `Movie` and `Genre` nodes and the `OF_GENRE` relationship between them.

In [12]:
memgraph.execute(
    """
    LOAD CSV FROM "/movies.csv" WITH HEADER AS row
    MERGE (m:Movie {id: toInteger(row.movieId), title: row.title}) 
    WITH m, row
    UNWIND split(row.genres, '|') as genre
    MERGE (m)-[:OF_GENRE]->(:Genre {name: genre});
    """
)

All that is left to create is `User` node and connect it to the `Movie` node that the user rated. We will again use `LOAD CSV` clause to import data from `ratings.csv` file.

In [13]:
memgraph.execute(
    """
    LOAD CSV FROM "/ratings.csv" WITH HEADER AS row
    MATCH (m:Movie {id: toInteger(row.movieId)})
    MERGE (u:User {id: toInteger(row.userId)})
    MERGE (u)-[:RATED {rating: toFloat(row.rating), timestamp: toFloat(row.timestamp)}]->(m);
    """
)

Great! We have imported all the data into Memgraph. Feel free to check it out in Memgraph Lab at `localhost:3000`.

## 8. Querying the database and retrieving results <a name="query"></a>

The simplest usage of the Cypher query language is to find data stored in the database. For that, you can use one of the following clauses:

- `MATCH` which searches for patterns.
- `WHERE` for filtering the matched data.
- `RETURN` for defining what will be presented to the user in the result set.
- `UNION` and `UNION ALL` for combining results from multiple queries.
- `UNWIND` for unwinding a list of values as individual rows.

Let's create a simple `MATCH` query to find all movies and order them by their title in ascending order.

In [14]:
results = memgraph.execute_and_fetch(
    """
    MATCH (m:Movie)
    RETURN m
    ORDER BY m.title ASC
    LIMIT 10;
    """
)

In [15]:
results = list(results)

for result in results:
    print(result["m"])


<Movie id=355218 labels={'Movie'} properties={'id': 117867, 'title': "'71 (2014)"}>
<Movie id=354632 labels={'Movie'} properties={'id': 97757, 'title': "'Hellboy': The Seeds of Creation (2004)"}>
<Movie id=352146 labels={'Movie'} properties={'id': 26564, 'title': "'Round Midnight (1986)"}>
<Movie id=352308 labels={'Movie'} properties={'id': 27751, 'title': "'Salem's Lot (2004)"}>
<Movie id=347232 labels={'Movie'} properties={'id': 779, 'title': "'Til There Was You (1997)"}>
<Movie id=355801 labels={'Movie'} properties={'id': 149380, 'title': "'Tis the Season for Love (2015)"}>
<Movie id=348155 labels={'Movie'} properties={'id': 2072, 'title': "'burbs, The (1989)"}>
<Movie id=348971 labels={'Movie'} properties={'id': 3112, 'title': "'night Mother (1986)"}>
<Movie id=353693 labels={'Movie'} properties={'id': 69757, 'title': '(500) Days of Summer (2009)'}>
<Movie id=351740 labels={'Movie'} properties={'id': 8169, 'title': '*batteries not included (1987)'}>


If you are wondering if these results have been correctly serialized back to the defined schema, you can check it out:

In [16]:
m = results[8]["m"]

print("Movie: ", m.title)
print("Type: ", type(m))


Movie:  (500) Days of Summer (2009)
Type:  <class '__main__.Movie'>


As expected, the movie in the result is an instance of `Movie` class and it has properties `id` and `title` that can be easily read.

If you're not that keen on Cypher query language, you can try out the GQLAlchemy **query builder**. Let's rewrite the above Cypher query with the help of the query builder:

In [17]:
from gqlalchemy import match

results_from_qb = (
    match()
    .node(labels="Movie", variable="m")
    .return_()
    .order_by("m.title ASC")
    .limit(10)
    .execute()
)
results_from_qb = list(results_from_qb)

for result in results_from_qb:
    print(result["m"])


<Movie id=355218 labels={'Movie'} properties={'id': 117867, 'title': "'71 (2014)"}>
<Movie id=354632 labels={'Movie'} properties={'id': 97757, 'title': "'Hellboy': The Seeds of Creation (2004)"}>
<Movie id=352146 labels={'Movie'} properties={'id': 26564, 'title': "'Round Midnight (1986)"}>
<Movie id=352308 labels={'Movie'} properties={'id': 27751, 'title': "'Salem's Lot (2004)"}>
<Movie id=347232 labels={'Movie'} properties={'id': 779, 'title': "'Til There Was You (1997)"}>
<Movie id=355801 labels={'Movie'} properties={'id': 149380, 'title': "'Tis the Season for Love (2015)"}>
<Movie id=348155 labels={'Movie'} properties={'id': 2072, 'title': "'burbs, The (1989)"}>
<Movie id=348971 labels={'Movie'} properties={'id': 3112, 'title': "'night Mother (1986)"}>
<Movie id=353693 labels={'Movie'} properties={'id': 69757, 'title': '(500) Days of Summer (2009)'}>
<Movie id=351740 labels={'Movie'} properties={'id': 8169, 'title': '*batteries not included (1987)'}>


As you can see, the result is the same as above. Next let's create a couple of example queries and check out the results.

### ✧ Find out how many users rated the movie with title 'Home Alone (1990)'

The correct Cypher query would be:

```
MATCH (u:User)-[:RATED]->(:Movie {title: 'Home Alone (1990)'})
RETURN count(u);
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [18]:
home_alone = (
    match()
    .node(labels="User", variable="u")
    .to("RATED")
    .node(labels="Movie", variable="m")
    .where("m.title", "=", "Home Alone (1990)")
    .return_({"count(u)": "num_of_users"})
    .execute()
)

results = list(home_alone)

for result in results:
    print(result["num_of_users"])

116


### ✧ Find out which movie is best rated

## 9. Exercises

<h3 align="center"><a href=#toc>⬆️ GO TO TOP ⬆️</a></h3> 